# Massachusetts income tax

Massachusetts has a mostly flat income tax of five percent, with personal exemptions based on filing status and number of dependents.
It also has other exemptions, deductions, and credits, including a partial federal Earned Income Tax Credit match.

## Examples

### Full income tax

In [8]:
from policyengine_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make_tax(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", age=25, rent=12_000)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=6)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MA")
    sim.vary("employment_income", max=100_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            state_income_tax=sim.calc("state_income_tax")[0].round(),
            ma_eitc=sim.calc("ma_eitc")[0].round(),
            ma_limited_income_tax_credit=sim.calc(
                "ma_limited_income_tax_credit"
            )[0].round(),
            mtr=sim.deriv(
                "state_income_tax", "employment_income", wrt_target="head"
            ),
            adults=adults,
            children=str(children),
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(0, 4):
        l.append(make_tax(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    state_income_tax="Massachusetts income tax",
    mtr="State marginal tax rate",
    adults="Adults",
    children="Children",
    state_eitc="Massachusetts EITC",
    ma_limited_income_tax_credit="Massachusetts Limited Income Credit",
)

fig = px.line(
    df,
    "employment_income",
    "state_income_tax",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts state income tax",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

Core marginal tax rates span from zero to five percent, but when including the EITC, they range from -13.5% to 12.3%, depending on income and household structure.

In [9]:
fig = px.line(
    df,
    "employment_income",
    "mtr",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts state income tax marginal tax rate",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()

### Massachusetts EITC

Massachusetts matches 30 percent of a filer's federal Earned Income Tax Credit.

In [10]:
def set_bounds(fig, x, y, buffer=1.1):
    fig.update_layout(
        xaxis_range=[0, x[y != 0].max() * buffer],
        yaxis_range=[0, y.max() * buffer],
    )


fig = px.line(
    df,
    "employment_income",
    "ma_eitc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts EITC",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
set_bounds(fig, df.employment_income, df.ma_eitc)
fig.show()

### Massachusetts Limited Income Credit

See [**Schedule NTS-L-NR/PY** No Tax Status and Limited Income Credit](https://www.mass.gov/doc/2021-schedule-nts-l-nrpy-no-tax-status-and-limited-income-credit/download).

In [11]:
fig = px.line(
    df,
    "employment_income",
    "ma_limited_income_tax_credit",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Massachusetts Limited Income Credit",
    color_discrete_map=COLOR_MAP,
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    yaxis_range=[0, df.ma_limited_income_tax_credit.max()],
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
set_bounds(fig, df.employment_income, df.ma_limited_income_tax_credit)
fig.show()